In [7]:
import os
import shutil
import pandas as pd 
from pathlib import Path
from foampilot import Meshing, Quantity, FluidMechanics, Solver, CSVFoamIntegrator

from foampilot.constant.constantDirectory import ConstantDirectory


In [8]:

current_path = Path. cwd()
case_path = Path. cwd() / "CoA_test_foampilot"
csv_inlet_data = current_path / "data" / "BPM120.csv"  
stl_path = current_path / "data"

case_path.mkdir(parents=True, exist_ok=True)

# 2. FLUID PROPERTIES
# Blood:  rho=1060, nu=3.77e-6
rho_blood = Quantity(1060, "kg/m^3")
nu_blood = Quantity(3.7735849056603773e-06, "m^2/s")

# 3. INITIALIZE SOLVER
solver = Solver(case_path)
solver.compressible = False
solver.with_gravity = False
solver.transient = True
solver.turbulence_model = "laminar"

# Set viscosity and density
solver.constant.transportProperties.rho = rho_blood
solver.constant.transportProperties.nu = nu_blood

🔔 Event: solver_change - Changing solver from None to incompressibleFluid
🔔 Event: solver_change - Changing solver from incompressibleFluid to incompressibleFluid
🔔 Event: solver_change - Changing solver from incompressibleFluid to incompressibleFluid
🔔 Event: solver_change - Changing solver from incompressibleFluid to incompressibleFluid
🔔 Event: solver_change - Changing solver from incompressibleFluid to incompressibleFluid


In [9]:

# 4. CONFIGURE CONTROLDICT & SYSTEM
solver.system.controlDict.startTime = 0.0
solver.system.controlDict.endTime = 0.5
solver.system.controlDict.deltaT = 1e-05
solver.system.controlDict.writeInterval = 0.01
solver.system.controlDict.writeControl = "adjustableRunTime"

solver.system.controlDict.libs = ["libmodularWKPressure.so"]

solver.system.controlDict.set_adaptive_time_step(maxCo=0.8)


In [10]:
# Paramètres PIMPLE avancés
solver.system.fvSolution.set_pimple(
    nOuterCorrectors=40,
    nCorrectors=3,
    nNonOrthogonalCorrectors=2,
    pRefPoint=(-0.013, -0.034, 0.001),
    momentumPredictor=True,
    turbOnFinalIterOnly=False,
    correctorResidualControl={
        "p": {"tolerance": 1e-1, "relTol": 0.1},
        "U": {"tolerance": 1e-2, "relTol": 0.1}
    },
    outerCorrectorResidualControl={
        "p": {"tolerance": 1e-2, "relTol": 0.1},
        "U": {"tolerance": 1e-3, "relTol": 0.1},
        "k": {"tolerance": 1e-3, "relTol": 0.1},
        "epsilon": {"tolerance": 1e-3, "relTol": 0.1},
        "omega": {"tolerance": 1e-3, "relTol": 0.1}
    }
)

# Facteurs de relaxation
solver.system.fvSolution.set_relaxation(
    fields={"p": 0.2},
    equations={"U": 0.3, "k": 0.2,"epsilon": 0.2, "omega": 0.2}
)

# Bloc cache
solver.system.fvSolution.set_cache(["grad(U)"])



In [11]:
from foampilot.constant.transportPropertiesFile import  NonNewtonianModels


bird_coeffs = {
    "nu0": Quantity(3.77e-6, "m^2/s"),
    "nuInf": Quantity(1e-3, "m^2/s"),
    "k": Quantity(0.01, "s"),
    "n": 0.6
}

solver.constant.transportProperties.set_non_newtonian(
    model="BirdCarreau",
    rho=rho_blood,
    **bird_coeffs
)

In [12]:

# Write system and constant folders
solver.system.write()
solver.constant.write()


[INFO] Constant directory written to /home/steven/foampilot/examples/coa/CoA_test_foampilot/constant


In [13]:
print(solver.constant.transportProperties.attributes)   

{'transportModel': 'BirdCarreau', 'rho': 1060.0, 'BirdCarreauCoeffs': {'nu0': 3.77e-06, 'nuInf': 0.001, 'k': 0.01, 'n': 0.6}}


In [14]:
from foampilot.utilities.manageunits import Quantity

# Définir les propriétés du sang
rho_blood = Quantity(1060, "kg/m^3")
nu_blood = Quantity(3.7735849056603773e-06, "m^2/s")

# Initialiser ConstantDirectory avec ces valeurs

# Modifier plus tard si nécessaire
solver.constant.transportProperties.nu = nu_blood
solver.constant.transportProperties.rho = rho_blood

print(solver.constant.transportProperties.nu.magnitude)   
print(solver.constant.transportProperties.rho.magnitude)  
print(solver.constant.transportProperties.attributes)     

# Écrire les fichiers constants
solver.constant.write()


[INFO] Constant directory written to /home/steven/foampilot/examples/coa/CoA_test_foampilot/constant


3.7735849056603773e-06
1060.0
{'transportModel': 'BirdCarreau', 'rho': Quantity(1060.0, 'kilogram / meter ** 3'), 'BirdCarreauCoeffs': {'nu0': 3.77e-06, 'nuInf': 0.001, 'k': 0.01, 'n': 0.6}}


In [15]:
# 5. MESH GENERATION (snappyHexMesh)
# Copy STL files from the original tutorial

stl_dest = case_path / "constant" / "triSurface"
stl_dest.mkdir(parents=True, exist_ok=True)

for stl_file in stl_path.glob("*.stl"):
    shutil.copy(stl_file, stl_dest)


# Initialize Meshing with snappy
mesh = Meshing(case_path, mesher="snappy")

snappy = mesh.mesher
snappy.stl_file = Path("wall_aorta.stl")

# Configure snappyHexMeshDict
snappy.locationInMesh = (-16.3177, -21.6838, -12.3357)
snappy.geometry = {
    "wall_aorta": {"type": "triSurfaceMesh", "file": "wall_aorta. stl", "name": "wall_aorta"},
    "inlet": {"type": "triSurfaceMesh", "file": "inlet.stl", "name":  "inlet"},
    "outlet1": {"type": "triSurfaceMesh", "file": "outlet1.stl", "name": "outlet1"},
    "outlet2": {"type": "triSurfaceMesh", "file": "outlet2.stl", "name": "outlet2"},
    "outlet3": {"type": "triSurfaceMesh", "file":  "outlet3.stl", "name": "outlet3"},
    "outlet4": {"type": "triSurfaceMesh", "file":  "outlet4.stl", "name": "outlet4"},
}
snappy.castellatedMeshControls["refinementSurfaces"] = {
    "wall_aorta": {"level": (0, 1)},
    "inlet": {"level": (0, 1)},
    "outlet1": {"level": (0, 1)},
    "outlet2": {"level": (0, 1)},
    "outlet3": {"level": (0, 1)},
    "outlet4": {"level": (0, 1)},
}
for part in ["wall_aorta", "inlet", "outlet1", "outlet2", "outlet3", "outlet4"]:
    snappy.add_feature(f"{part}. eMesh", 1)
stl_files = ["wall_aorta.stl", "inlet.stl", "outlet1.stl", "outlet2.stl", "outlet3.stl", "outlet4.stl"]

snappy.addLayers = True
snappy.add_layer("wall_aorta", 3)
snappy.addLayersControls["finalLayerThickness"] = 0.3

# Write snappyHexMeshDict
mesh.write()

# Generate surfaceFeaturesDict
snappy.write_surface_features_dict(stl_list=stl_files, included_angle=30)



snappyHexMeshDict written to /home/steven/foampilot/examples/coa/CoA_test_foampilot/system/snappyHexMeshDict
surfaceFeaturesDict written to /home/steven/foampilot/examples/coa/CoA_test_foampilot/system/surfaceFeaturesDict
surfaceFeaturesDict written to /home/steven/foampilot/examples/coa/CoA_test_foampilot/system/surfaceFeaturesDict


In [16]:

snappy.run()

Error running snappyHexMesh:


--> FOAM FATAL ERROR: 
Cannot find file "points" in directory "polyMesh" in times "0" down to constant

    From function virtual Foam::IOobject Foam::fileOperation::findInstance(const Foam::IOobject&, Foam::scalar, const Foam::word&) const
    in file global/fileOperations/fileOperation/fileOperation.C at line 860.

FOAM exiting




In [15]:

def setup_coa_case():
    # 1. DEFINE CASE PATH
    current_path = Path. cwd()
    case_path = Path. cwd() / "CoA_test_foampilot"
    csv_inlet_data = current_path / "data" / "BPM120.csv"  
    stl_path = current_path / "data"

    case_path.mkdir(parents=True, exist_ok=True)

    # 2. FLUID PROPERTIES
    # Blood:  rho=1060, nu=3.77e-6
    rho_blood = Quantity(1060, "kg/m^3")
    nu_blood = Quantity(3.7735849056603773e-06, "m^2/s")

    # 3. INITIALIZE SOLVER
    solver = Solver(case_path)
    solver.compressible = False
    solver.with_gravity = False
    solver.transient = True
    solver.turbulence_model = "laminar"
    
    # Set viscosity and density
    solver.constant. transportProperties.nu = nu_blood
    solver. constant.transportProperties.rho = rho_blood

    # 4. CONFIGURE CONTROLDICT & SYSTEM
    solver.system.controlDict.application = "foamRun"
    solver.system. controlDict.solver = "incompressibleFluid"
    solver.system. controlDict.startTime = 0.0
    solver.system.controlDict.endTime = 0.5
    solver.system.controlDict.deltaT = 1e-05
    solver.system.controlDict.writeInterval = 0.01
    solver.system.controlDict.adjustTimeStep = True
    solver. system.controlDict.maxCo = 0.8
    solver.system.controlDict.libs = ("libmodularWKPressure.so",)
    
    # Configure PIMPLE for CoA_test
    solver.system.fvSolution. PIMPLE.update({
        "nOuterCorrectors": 40,
        "nCorrectors": 2,
        "nNonOrthogonalCorrectors":  1,
        "residualControl": {
            "p": 0.01,
            "U": 0.001
        }
    })
    
    # Write system and constant folders
    solver.system.write()
    solver.constant.write()

    # 5. MESH GENERATION (snappyHexMesh)
    # Copy STL files from the original tutorial
    
    stl_dest = case_path / "constant" / "triSurface"
    stl_dest.mkdir(parents=True, exist_ok=True)
    
    for stl_file in stl_path.glob("*.stl"):
        shutil.copy(stl_file, stl_dest)
    

    # Initialize Meshing with snappy
    mesh = Meshing(case_path, mesher="snappy")
    snappy = mesh.mesher
    snappy.stl_file = Path("wall_aorta.stl")
    
    # Configure snappyHexMeshDict
    snappy.locationInMesh = (-16.3177, -21.6838, -12.3357)
    snappy.geometry = {
        "wall_aorta": {"type": "triSurfaceMesh", "file": "wall_aorta. stl", "name": "wall_aorta"},
        "inlet": {"type": "triSurfaceMesh", "file": "inlet.stl", "name":  "inlet"},
        "outlet1": {"type": "triSurfaceMesh", "file": "outlet1.stl", "name": "outlet1"},
        "outlet2": {"type": "triSurfaceMesh", "file": "outlet2.stl", "name": "outlet2"},
        "outlet3": {"type": "triSurfaceMesh", "file":  "outlet3.stl", "name": "outlet3"},
        "outlet4": {"type": "triSurfaceMesh", "file":  "outlet4.stl", "name": "outlet4"},
    }
    snappy.castellatedMeshControls["refinementSurfaces"] = {
        "wall_aorta": {"level": (0, 1)},
        "inlet": {"level": (0, 1)},
        "outlet1": {"level": (0, 1)},
        "outlet2": {"level": (0, 1)},
        "outlet3": {"level": (0, 1)},
        "outlet4": {"level": (0, 1)},
    }
    for part in ["wall_aorta", "inlet", "outlet1", "outlet2", "outlet3", "outlet4"]:
        snappy.add_feature(f"{part}. eMesh", 1)
    stl_files = ["wall_aorta.stl", "inlet.stl", "outlet1.stl", "outlet2.stl", "outlet3.stl", "outlet4.stl"]

    snappy.addLayers = True
    snappy.add_layer("wall_aorta", 3)
    snappy.addLayersControls["finalLayerThickness"] = 0.3
    
    # Write snappyHexMeshDict
    mesh.write()

    # Generate surfaceFeaturesDict
    snappy.write_surface_features_dict(case_path, stl_files, included_angle=30)
    snappy.write_snappyHexMesh_dict(case_path)
    snappy.run()

    # Initialize boundaries
    solver.boundary.initialize_boundary()
    
    # Define custom BCs for Windkessel
    inlet_bc_u = {
        "type": "timeVaryingMappedFixedValue",
        "offset": "(0 0 0)",
        "setAverage": "false"
    }
    inlet_bc_p = {"type": "zeroGradient"}
    
    wk_p_bc = {
        "type":  "modularWKPressure",
        "phi": "phi",
        "order": "2",
        "R": "1000",
        "C": "1e-06",
        "Z": "100",
        "p0": "10666",
        "value": "uniform 10666"
    }
    wk_u_bc = {
        "type": "stabilizedWindkesselVelocity",
        "beta": "1.0",
        "enableStabilization": "true",
        "value": "uniform (0 0 0)"
    }
    
    # Apply to patches
    solver.boundary.fields["U"]["inlet"] = inlet_bc_u
    solver.boundary.fields["p"]["inlet"] = inlet_bc_p
    
    for i in range(1, 5):
        patch = f"outlet{i}"
        solver.boundary.fields["U"][patch] = wk_u_bc
        solver.boundary.fields["p"][patch] = wk_p_bc
        
    solver.boundary.fields["U"]["wall_aorta"] = {"type": "noSlip"}
    solver.boundary. fields["p"]["wall_aorta"] = {"type": "zeroGradient"}

    # Write all boundary files (this creates the 0/ directory files)
    solver.boundary.write_boundary_conditions()
    
    # SECTION BOUNDARYDATA 
    # Charger le CSV
    df_csv = pd.read_csv(csv_inlet_data)  # Colonnes: Time, Flowrate

    # Initialiser l'intégrateur et charger le maillage
    integrator = CSVFoamIntegrator(case_path)
    integrator.load_mesh()  
    df_patch = integrator.get_patch_dataframe("inlet")  # Patch d'entrée

    # Répliquer le CSV pour chaque point du patch
    df_full = []
    for _, row in df_csv.iterrows():
        for _, p in df_patch.iterrows():
            df_full.append({
                "time": row["Time"],
                "x": p["x"],
                "y":  p["y"],
                "z": p["z"],
                "Flowrate": row["Flowrate"]  
            })
    df_full = pd.DataFrame(df_full)

    # ✅ Exporter en boundaryData
    integrator.export_to_boundary_data("inlet", df_full, "Flowrate", time="0")
